In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import numpy as np
import nltk

In [3]:
def get_reviews(path, positive=True):
    label = 1 if positive else 0
    
    with open(path, 'r') as f:
        review_text = f.readlines()
        
    reviews = []
    for text in review_text:
        # Return a tuple of the review text and a label for whether it
        # is a positive or negative review
        reviews.append((text, label))
    return reviews

In [9]:
def extract_reviews():
    positive_reviews = get_reviews("rt-polaritydata/rt-polarity.pos", positive=True)
    negative_reviews = get_reviews("rt-polaritydata/rt-polarity.neg", positive=False)
    
    return positive_reviews, negative_reviews

In [10]:
positive_reviews, negative_reviews = extract_reviews()

In [11]:
len(positive_reviews)

5331

In [12]:
len(negative_reviews)

5331

In [13]:
positive_reviews[:2]

[('the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . \n',
  1),
 ('the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth . \n',
  1)]

In [14]:
TRAIN_DATA = 5000
TOTAL_DATA = len(positive_reviews)

train_reviews = positive_reviews[:TRAIN_DATA] + negative_reviews[:TRAIN_DATA]

test_positive_reviews = positive_reviews[TRAIN_DATA:TOTAL_DATA]
test_negative_reviews = negative_reviews[TRAIN_DATA:TOTAL_DATA]

In [18]:
def get_vocabulary(train_reviews):
    words_set = set()
    
    for review in train_reviews:
        words_set.update(review[0].split())
        
    return list(words_set)

vocabulary = get_vocabulary(train_reviews)

In [19]:
len(vocabulary)

20728

In [20]:
vocabulary[:5]

['believer', 'urge', 'heal', 'smartly', 'oozes']

In [21]:
def extract_features(review_text):
    
    # Split the review into words, and create a set of the words
    review_words = set(review_text.split())
    
    features = {}
    for word in vocabulary:
        features[word] = (word in review_words)
        
    return features

In [22]:
train_features = nltk.classify.apply_features(extract_features, train_reviews)

In [23]:
trained_classifier = nltk.NaiveBayesClassifier.train(train_features)

In [24]:
def sentiment_calculator(review_text):
    features = extract_features(review_text)
    return trained_classifier.classify(features)

In [25]:
sentiment_calculator("What an amazing movie!")

1

In [26]:
sentiment_calculator("What a terrible movie")

0

In [27]:
def classify_test_reviews(test_positive_reviews, test_negative_reviews, sentiment_calculator):
    
    positive_results = [sentiment_calculator(review[0]) for review in test_positive_reviews]
    negative_results = [sentiment_calculator(review[0]) for review in test_negative_reviews]
    
    true_positives = sum(x > 0 for x in positive_results)
    true_negatives = sum(x == 0 for x in negative_results)
    
    percent_true_positive = float(true_positives) / len(positive_results)
    percent_true_negative = float(true_negatives) / len(negative_results)
    
    total_accurate = true_positives + true_negatives
    total = len(positive_results) + len(negative_results)
    
    print("Accuracy on posititve reviews = " +"%.2f" % (percent_true_positive * 100) + "%")
    print("Accuracy on negative reviews = " +"%.2f" % (percent_true_negative * 100) + "%")
    print("Overall accuracy = " +"%.2f" % (total_accurate * 100/ total) + "%")

In [28]:
classify_test_reviews(test_positive_reviews, test_negative_reviews, sentiment_calculator)

Accuracy on posititve reviews = 78.25%
Accuracy on negative reviews = 80.66%
Overall accuracy = 79.46%
